In [84]:
from kubeflow.metadata import metadata as kubeflow_metadata
import pandas as pd
import tensorflow_model_analysis as tfma
import subprocess

In [77]:
meta_pod = subprocess.run(["kubectl","describe","services","metadata-grpc-service"],capture_output=True).stdout

endpoint_addr = [x for x in meta_pod.decode('utf-8').split('\n') if 'Endpoints' in x][0].replace('Endpoints:','').strip()

METADATA_STORE_HOST, METADATA_STORE_PORT = endpoint_addr.split(':')

In [82]:
ws = kubeflow_metadata.Workspace(store=kubeflow_metadata.Store(grpc_host=METADATA_STORE_HOST, grpc_port=int(METADATA_STORE_PORT)),
name='workspace_1',description='a workspace for testing',labels={"n1":"v1"})

eval_artifacts = ws.store.get_artifacts_by_type('ModelEvaluation')

print([(x.id, x.uri) for x in eval_artifacts])

eval_results = tfma.load_eval_results([x.uri for x in eval_artifacts])
# eval_artifacts

[(23, 'gs://dst-pjm-pipeline-20200827/tfx_pipeline_output/pjm-pipeline-20200909a/Evaluator/evaluation/43'), (27, 'gs://dst-pjm-pipeline-20200827/tfx_pipeline_output/pjm-pipeline-20200909a/Evaluator/evaluation/51'), (38, 'gs://dst-mlpipes-kubeflowpipelines-default/tfx_iris/edb0c03f-edb1-4620-90d9-c58f95eefc89/Evaluator/evaluation/60'), (122, 'gs://pjm-pipeline-artifacts/OL653374_kfp/202010061810/Evaluator/evaluation/170'), (133, 'gs://pjm-pipeline-artifacts/Evaluator/evaluation/184')]


In [85]:
def trim_location(result):
    location_str = result._asdict()['model_location']
    return "_".join(location_str.split('/')[3:5])

def process_metrics(result):
    metric_list = result._asdict()['slicing_metrics']
    metric_dict = metric_list[0][1]['']['']
    return pd.DataFrame.from_dict(metric_dict)


result_df = pd.concat({
    trim_location(x):process_metrics(x) 
    for x in eval_results.get_results()})
result_df

,,binary_accuracy,binary_accuracy_diff,sparse_categorical_accuracy
tfx_pipeline_output_pjm-pipeline-20200909a,doubleValue,0.711934,NaN,NaN
,doubleValue,0.920047,0.208113,NaN
tfx_iris_edb0c03f-edb1-4620-90d9-c58f95eefc89,doubleValue,NaN,NaN,0.513514
OL653374_kfp_202010061810,doubleValue,0.547361,NaN,NaN
Trainer_model,doubleValue,0.313326,NaN,NaN
